# HRRR NOAA Temperature data

In [1]:
from herbie import Herbie

In [2]:
import io
from datetime import date, timedelta

import xarray as xr
import requests

from pathlib import Path
import pandas as pd
import geopy.distance as distance
import numpy as np
from tqdm import tqdm
import json

In [3]:
DATA_DIR = Path.cwd().resolve() / "data"
SENTINEL_DATA_DIR = DATA_DIR / "sentinel"

In [4]:
# get our bounding box to search latitude and longitude coordinates
def get_bounding_box(latitude, longitude, meter_buffer=50000):
    """
    Given a latitude, longitude, and buffer in meters, returns a bounding
    box around the point with the buffer on the left, right, top, and bottom.

    Returns a list of [minx, miny, maxx, maxy]
    """
    distance_search = distance.distance(meters=meter_buffer)

    # calculate the lat/long bounds based on ground distance
    # bearings are cardinal directions to move (south, west, north, and east)
    min_lat = distance_search.destination((latitude, longitude), bearing=180)[0]
    min_long = distance_search.destination((latitude, longitude), bearing=270)[1]
    max_lat = distance_search.destination((latitude, longitude), bearing=0)[0]
    max_long = distance_search.destination((latitude, longitude), bearing=90)[1]

    return [min_long, min_lat, max_long, max_lat]

In [5]:
metadata = pd.read_csv(DATA_DIR / "metadata.csv")

In [6]:
sentinel_data = json.load(open(SENTINEL_DATA_DIR / "selected_items.txt"))

In [7]:
train_subset = metadata[metadata.uid.isin(sentinel_data.keys())]

In [8]:
temp_dict = json.load(open(SENTINEL_DATA_DIR / f"temperatures.txt"))

In [9]:
# Constants for creating the full URL
forecast_hour = 0   # offset from cycle time
product = 'sfc' #"wrfsfcf" # 2D surface levels
i=0
for row in tqdm(train_subset.itertuples(), total=len(train_subset)):
    pass
    if temp_dict.get(row.uid) == None:
        date = row.date
        [min_long, min_lat, max_long, max_lat] = get_bounding_box(row.latitude, row.longitude, 2000)
        try:
            H = Herbie(
                date,  # model run date
                model="hrrr",  # model name
                product=product,  # model produce name (model dependent)
                fxx=forecast_hour,  # forecast lead time
            )
            ds = H.xarray(":TMP:surface", remove_grib=True)

            temp_dict[row.uid] = float(ds.t.where(ds.latitude >= min_lat).where(ds.latitude <= max_lat).where(ds.longitude <= max_long+360).where(ds.longitude >= min_long+360).mean(skipna=True).values)
        except:
            temp_dict[row.uid] = -1
        if i % 100 == 0:
            with open(SENTINEL_DATA_DIR / "temperatures.txt", "w") as f:
                json.dump(temp_dict, f)
        i += 1


 86%|████████▋ | 11898/13763 [00:00<00:00, 118270.37it/s]

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2017-Aug-28 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2018-Aug-28 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2018-Nov-07 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


 91%|█████████ | 12504/13763 [00:20<00:02, 442.61it/s]   

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2021-Aug-17 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2019-Jun-25 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2021-Aug-11 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


c:\Users\Usuario\anaconda3\envs\tfm\Lib\site-packages\herbie\core.py:1052: UserWarning: Will not remove GRIB file because it previously existed.
  warnings.warn("Will not remove GRIB file because it previously existed.")


...

100%|█████████▉| 13761/13763 [1:34:54<00:07,  3.64s/it]

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2017-Jun-19 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


100%|█████████▉| 13762/13763 [1:34:57<00:03,  3.60s/it]

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2016-Nov-21 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


100%|██████████| 13763/13763 [1:35:01<00:00,  2.41it/s]


In [10]:
with open(SENTINEL_DATA_DIR / "temperatures.txt", "w") as f:
    json.dump(temp_dict, f)